In [1]:
import os
import sys
import pandas as pd
import cv2
import numpy as np

# Setup paths to your local autocnet and plio enviorment (remove if you used conda install)
sys.path.insert(0, "/Users/thatcher/Documents/Work/autocnet")
sys.path.insert(0, "/Users/thatcher/Documents/Work/plio")

from autocnet.examples import get_path
from autocnet.graph.network import CandidateGraph
from IPython.display import display
from autocnet.camera import camera
from matplotlib import pyplot as plt
from autocnet.transformation.fundamental_matrix import compute_fundamental_matrix
from autocnet.transformation import trifocal as tf
from autocnet.transformation.fundamental_matrix import compute_epipolar_lines

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# #Point to the adjacency Graph
# adjacency = get_path('three_image_adjacency.json')
# basepath = get_path('Apollo15')
# cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

# #Apply SIFT to extract features
# cg.extract_features(extractor_method='sift', extractor_parameters={'nfeatures':500})

# #Match
# cg.match()

# #Apply outlier detection
# cg.symmetry_checks()
# cg.ratio_checks()

# #Compute a homography and apply RANSAC
# cg.compute_fundamental_matrices(clean_keys=['ratio', 'symmetry'])

In [3]:
# cg.plot()

In [4]:
# cg.node[1]['data'].plot()

In [5]:
# ax1 = plt.subplot(1,1,1)
# ax = cg.node[0]['data'].plot(ax=ax1, color='y')

In [6]:
# fig, ax = plt.subplots(1,1)
# print(ax)
# ax = cg.edges[0,1]['data'].plot(clean_keys=['ratio', 'symmetry', 'fundamental'], ax=ax)

In [7]:
# fig, ax = plt.subplots(1,1)
# cg.edges[0,1]['data'].plot_source(clean_keys=['ratio', 'symmetry', 'fundamental'], ax=ax)

In [8]:
# fig, ax = plt.subplots(1,1)
# cg.edges[0,1]['data'].plot(node='d', clean_keys=['ratio', 'symmetry', 'fundamental'])

In [9]:
# cg.edges[0,1]['data'].plot(clean_keys=['ratio', 'symmetry', 'fundamental'], line_kwargs={'linewidth':0})

In [10]:
# e1 = cg.edges[0,1]['data'].get_matches(clean_keys=['fundamental'])
# e2 = cg.edges[0,2]['data'].get_matches(clean_keys=['fundamental'])
# e3 = cg.edges[1,2]['data'].get_matches(clean_keys=['fundamental'])

In [11]:
# e1_s = e1[['source_x', 'source_y']]
# e1_d = e1[['destination_x', 'destination_y']]

# e2_s = e2[['source_x', 'source_y']]
# e2_d = e2[['destination_x', 'destination_y']]

# e3_s = e3[['source_x', 'source_y']]
# e3_d = e3[['destination_x', 'destination_y']]

In [12]:
# im1 = cg.node[0]['data'].geodata.read_array()
# im2 = cg.node[1]['data'].geodata.read_array()
# im3 = cg.node[2]['data'].geodata.read_array()

In [13]:
# imshow(im1, cmap='Greys')
# plot(e1.source_x, e1.source_y, 'ro')

In [14]:
# f01 = cg.edges[0,1]['data']['fundamental_matrix']
# f02 = cg.edges[0,2]['data']['fundamental_matrix']

# Makes homogenous matrix values for Z
# e1['int'] = 1
# e2['int'] = 1
# e3['int'] = 1

# # Create the subset dataframe
# e1 = e1[['source_x', 'source_y', 'int']]
# e2 = e2[['source_x', 'source_y', 'int']]
# e3 = e3[['source_x', 'source_y', 'int']]

# Loads dataframe to make them a matrix
# matrix_e1 = (np.asarray(e1))
# matrix_e2 = (np.asarray(e2))
# matrix_e3 = (np.asarray(e3))

In [165]:
from IPython.core.display import display, HTML
display(HTML('<h1>Functions to get correct lines to estimate points</h1>'))

In [15]:
# Takes a single line el, and point x
def get_correct_line(el, x):
    '''
    Inputs:

    e1: Epipolar line from image-3 to image-1 or image-2 to image-1
    x: Point from image that was not in the epipolar image (point to estimate)  

    Returns:
    Perpindicular based on the epipolar line and point
    '''
    correct_line = np.array([el[1], -el[0], -x[0]*el[1] + x[1]*el[0]]).T
    return correct_line

def get_lines(points, F):
    '''
    Inputs:

    points: Homogeneous points from images that matches
    F: Fundamental matrix from 2nd or 3rd image to first

    Returns:

    Epipolar Line based on the fundamantal matrix and match points
    '''
    line = compute_epipolar_lines(F, points)
    return line

In [166]:
'''Test Point to get Trifocal Tensor Working'''

# -1 from simple_matrix2 points to use for testing
simple_matrix1 = np.array([[0,0,1], [0,1,1], [0,2,1], [0,3,1], [0,4,1], [0,5,1],
                           [0,6,1], [0,7,1], [1.5,1.5,1], [3,0,1], [4,0,1], [3,3,1], [4,4,1]])

# Test points to use for testing
simple_matrix2 = np.array([[1,1,1], [1,2,1], [1,3,1], [1,4,1], [1,5,1], [1,6,1],
                           [1,7,1], [1,8,1], [2.5,2.5,1], [4,1,1], [5,1,1], [4,4,1], [5,5,1]])

# +1 from simple_matrix2 points to use for testing
simple_matrix3 = np.array([[2,2,1], [2,3,1], [2,4,1], [2,5,1], [2,6,1], [2,7,1],
                           [2,8,1], [2,9,1], [3.5,3.5,1], [5,2,1], [6,2,1], [5,5,1], [6,6,1]])

# Build DataFrames from simple points
df_1 = pd.DataFrame(simple_matrix1)
df_2 = pd.DataFrame(simple_matrix2)
df_3 = pd.DataFrame(simple_matrix3)

In [171]:
from IPython.core.display import display, HTML
display(HTML('<h1>Estimates point from point correspondents</h1><p>This works with 3 guaranteed matches between 3 images</p>'))

In [168]:
'''Setup for correct lines and runs the functions to estimate points in 3rd image'''

# Init tensor
trifocal = tf.Trifocal(simple_matrix1.T, simple_matrix2.T, simple_matrix3.T)

# These work for fundamental
f_21, mask = compute_fundamental_matrix(df_2, df_1)
f_31, mask = compute_fundamental_matrix(df_3, df_1)

# Epipolar lines 21/31
line21 = get_lines(simple_matrix1, f_21)
line31 = get_lines(simple_matrix1, f_31)

index = 0

# Gets the proper line to estimate points
correct_line21 = get_correct_line(line21[index], simple_matrix2[index])
correct_line31 = get_correct_line(line31[index], simple_matrix2[index])

# Calculate point in 3rd image from point in 1st and line in 2nd transfer method
# These get the correct results
p3 = trifocal.pl_(simple_matrix1[index], correct_line21)
p2 = trifocal.pl_(simple_matrix1[index], correct_line31)

# Convert back to homogeneous
p3_homo = np.array([p3[0]/p3[-1], p3[1]/p3[-1], p3[2]/p3[-1]])
p2_homo = np.array([p2[0]/p2[-1], p2[1]/p2[-1], p2[2]/p2[-1]])

# Estimate of point3
print('\nEstimated Point3: \n', p3_homo.T)
print('\nEStimated Point2: \n', p2_homo.T)
print('\nOrig: \n', simple_matrix3[index])

'''Plot points from matches'''
# imshow(im2, cmap='Greys')
# plot(p3_refined[0], p3_refined[1], 'ro')


Estimated Point3: 
 [6. 6. 1.]

EStimated Point2: 
 [6. 6. 1.]

Orig: 
 [6. 6. 1.]


/Users/thatcher/Documents/Work/autocnet/autocnet/transformation/fundamental_matrix.py:341: UserWarning: Unable to apply MLE.  Not enough correspondences.  Returning with a RANSAC computed F matrix.
  warnings.warn("Unable to apply MLE.  Not enough correspondences.  Returning with a RANSAC computed F matrix.")


In [176]:
from IPython.core.display import display, HTML
display(HTML('<h1>Estimates point from camera matricies</h1>'))
display(HTML('This does not work, the calculation of the camera matrix is wrong.'))
display(HTML('<p>The tensor from the above cell should be the same or close to the same.</p>'))

In [177]:
# -1 from simple_matrix2 points to use for testing
simple_matrix1 = np.array([[0,0,1], [0,1,1], [0,2,1], [0,3,1], [0,4,1], [0,5,1],
                           [0,6,1], [0,7,1], [1.5,1.5,1], [6,0,1], [5,0,1], [4,0,1],
                           [3,0,1], [4,0,1], [3,3,1], [4,4,1]])

# Test points to use for testing
simple_matrix2 = np.array([[1,1,1], [1,2,1], [1,3,1], [1,4,1], [1,5,1], [1,6,1],
                           [1,7,1], [1,8,1], [2.5,2.5,1], [7,1,1], [6,1,1], [5,1,1], [4,1,1], 
                           [5,1,1], [4,4,1], [5,5,1]])

# +1 from simple_matrix2 points to use for testing
simple_matrix3 = np.array([[2,2,1], [2,3,1], [2,4,1], [2,5,1], [2,6,1], [2,7,1],
                           [2,8,1], [2,9,1], [3.5,3.5,1], [8,2,1], [7,2,1], [6,2,1], [5,2,1], 
                           [6,2,1], [5,5,1], [6,6,1]])

# Build DataFrames from simple points
df_1 = pd.DataFrame(simple_matrix1)
df_2 = pd.DataFrame(simple_matrix2)
df_3 = pd.DataFrame(simple_matrix3)

# These work for fundamental
f_21, mask = compute_fundamental_matrix(df_2, df_1)
f_31, mask = compute_fundamental_matrix(df_3, df_1)
f_32, mask = compute_fundamental_matrix(df_3, df_2)

# Computes the camera matricies for each fundamental
camera21 = camera.estimated_camera_from_f(f_21)
camera31 = camera.estimated_camera_from_f(f_31)
camera32 = camera.estimated_camera_from_f(f_32)

# Unclear if this works
trifocal2 = tf.Trifocal(camera_21[1][:3], camera_31[0][:3])

print('Correct:\n', trifocal)
print('\nNot Correct:\n',trifocal2)

# Epipolar lines 21/31
line21 = get_lines(simple_matrix1, f_21)
line31 = get_lines(simple_matrix1, f_31)

index = 0

# Gets the proper line to estimate points
correct_line21 = get_correct_line(line21[index], simple_matrix2[index])
correct_line31 = get_correct_line(line31[index], simple_matrix2[index])

# Calculate point in 3rd image from point in 1st and line in 2nd transfer method
# These get the correct results
p3 = trifocal2.pl_(simple_matrix1[index], correct_line21)
p2 = trifocal2.pl_(simple_matrix1[index], correct_line31)

# Convert back to homogeneous
p3_homo = np.array([p3[0]/p3[-1], p3[1]/p3[-1], p3[2]/p3[-1]])
p2_homo = np.array([p2[0]/p2[-1], p2[1]/p2[-1], p2[2]/p2[-1]])

# Estimate of point3
print('\nEstimated Point3: \n', p3_homo.T)
print('\nEStimated Point2: \n', p2_homo.T)
print('\nOrig: \n', simple_matrix3[index])

Correct:
 [[[-5.01043455e-01 -9.95020006e-02 -3.81612786e-02]
  [ 1.15897217e-01 -1.50162001e-16  9.97465999e-18]
  [ 1.06851580e-01  6.50521303e-17  4.16333634e-17]]

 [[ 4.14165230e-17 -3.19424036e-02  6.28837260e-17]
  [-4.69101051e-01  1.63952160e-02 -3.81612786e-02]
  [-5.89805982e-17  1.06851580e-01  1.38777878e-17]]

 [[-5.32985858e-01 -1.63386808e-01 -7.01036822e-02]
  [-2.37306618e-01  1.32292433e-01  7.77359380e-02]
  [-2.55397891e-01  1.14201160e-01  6.86903016e-02]]]

Not Correct:
 [[[-0.5401021  -0.13641212 -0.05112331]
  [-0.24024885  0.02831885  0.00652929]
  [-0.23539798  0.02316018  0.00488891]]

 [[-0.07956547  0.13553941 -0.13503819]
  [-0.67169891 -0.69549002  0.4118342 ]
  [ 0.70063662 -0.36123896  0.48706549]]

 [[-0.6538829  -0.0063827   0.0988621 ]
  [-0.40375867 -0.56571858 -0.58455094]
  [-0.50203986 -0.54909582 -0.54948617]]]

Estimated Point3: 
 [-0.17951451  0.84479871  1.        ]

EStimated Point2: 
 [-0.14031887  0.85033799  1.        ]

Orig: 
 [2. 2. 1

In [ ]:


# size = 50
# for i, r in e1.iterrows():

#     fig, (ax1, ax2, ax3) = plt.subplots(1,3)
#     ax1.imshow(im1[int(r.source_y-size):int(r.source_y+size), int(r.source_x-size):int(r.source_x+size)], cmap='Greys')
#     ax2.imshow(im2[int(r.destination_y-size):int(r.destination_y+size), int(r.destination_x-size):int(r.destination_x+size)], cmap='Greys')
#     ax3.imshow(im3[:,:], cmap='Greys')
#     show()

In [ ]:
# from autocnet.matcher.suppression_funcs import distance

In [ ]:
# cg.edges[0,1]['data'].suppress(k=50, suppression_func=distance, clean_keys=['fundamental'])

# # Plot, in blue the points that passed all outlier detectors so far
# cg.edges[0,1]['data'].plot(clean_keys=['ratio', 'symmetry', 'fundamental'], line_kwargs={'linewidth':0})
# # Overlay, in red, the points that remain after suppression
# cg.edges[0,1]['data'].plot(clean_keys=['suppression'], line_kwargs={'linewidth':0}, scatter_kwargs={'color':'red'})

In [ ]:

# cg.edges[0,1]['data'].suppress(k=25, suppression_func=distance, clean_keys=['fundamental'])

# # Plot, in blue the points that passed all outlier detectors so far
# cg.edges[0,1]['data'].plot(clean_keys=['ratio', 'symmetry', 'fundamental'], line_kwargs={'linewidth':0})
# # Overlay, in red, the points that remain after suppression
# cg.edges[0,1]['data'].plot(clean_keys=['suppression'], line_kwargs={'linewidth':0}, scatter_kwargs={'color':'red'})

In [ ]:
# cg.edges[0,1]['data'].suppress(k=10, suppression_func=distance, clean_keys=['fundamental'])

# # Plot, in blue the points that passed all outlier detectors so far
# cg.edges[0,1]['data'].plot(clean_keys=['ratio', 'symmetry', 'fundamental'], line_kwargs={'linewidth':0})
# # Overlay, in red, the points that remain after suppression
# cg.edges[0,1]['data'].plot(clean_keys=['suppression'], line_kwargs={'linewidth':0}, scatter_kwargs={'color':'red'})